In [6]:
from automated_statistician import greedy_search
import GPy
import numpy as np
import matplotlib.pyplot as plt
import utils
from functools import partial
from utils import KernelFunction, KernelEnvironment, log_likelihood_reward
from utils import plot_kernel_function, compare_kernels
from gflownet import GFlowNet 
import torch.nn.functional as F
from torch.distributions import Categorical
import torch
from utils import ForwardPolicy, BackwardPolicy
import random
from utils import train

import evaluation
from evaluation import create_random_kernel

In [7]:
ll = -1
while ll < 0:
    true_kernel = create_random_kernel()
    X, Y, true_kernel_str = utils.generate_gp_data(true_kernel, input_dim=1, n_points=30, noise_var=1e-4)
    X_test, Y_test, _ = utils.generate_gp_data(true_kernel, input_dim=1, n_points=30, noise_var=1e-4)
    ll = utils.evaluate_likelihood(true_kernel, X, Y, runtime=False)

print("True Kernel:", true_kernel_str, "Log Marginal Likelihood:", utils.evaluate_likelihood(true_kernel, X, Y, runtime=False))
print("True Kernel:", true_kernel_str, "Log Marginal Likelihood:", utils.evaluate_likelihood(true_kernel, X_test, Y_test, runtime=False))

  -> Randomizing 'Constant' params...
  -> Randomizing 'Constant' params...
  -> Randomizing 'Linear' params...
True Kernel: ((Constant({'variance': 1.185}) * Constant({'variance': 0.849})) * Linear({'variances': 1.288})) Log Marginal Likelihood: 90.96786511394015
True Kernel: ((Constant({'variance': 1.185}) * Constant({'variance': 0.849})) * Linear({'variances': 1.288})) Log Marginal Likelihood: 91.86569782585374


In [16]:
ll_test = utils.evaluate_likelihood(true_kernel, X_test, Y_test, runtime=False)
rmse_test = evaluation.calculate_rmse(true_kernel, X, Y, X_test, Y_test)
    
print("True Kernel:", str(true_kernel), "Log Marginal Likelihood:", ll_test, "RMSE:", rmse_test)
print("No std (0.0) in the true kernel")
print("===========================================")

greedy_kernel, _ = greedy_search(X, Y, method='BIC', max_steps=5)

ll_test = utils.evaluate_likelihood(greedy_kernel, X_test, Y_test, runtime=False)
rmse_test = evaluation.calculate_rmse(greedy_kernel, X, Y, X_test, Y_test)
    
print("Greedy Search Kernel:", str(greedy_kernel), "Log Marginal Likelihood:", ll_test, "RMSE:", rmse_test)
print("No std (0.0) in the greedy kernel")
print("===========================================")

rbf_kernel = KernelFunction().rbf()
linear_kernel = KernelFunction().linear()
periodic_kernel = KernelFunction().periodic()
constant_kernel = KernelFunction().constant()

kernels = [rbf_kernel, linear_kernel, periodic_kernel, constant_kernel]
for kernel in kernels:
    ll_test = utils.evaluate_likelihood(kernel, X_test, Y_test, runtime=False)
    rmse_test = evaluation.calculate_rmse(kernel, X, Y, X_test, Y_test)
    
    print(" Basis kernel:", str(kernel), "Log Marginal Likelihood:", ll_test, "RMSE:", rmse_test)
    print("No std (0.0) in the kernel")
print("===========================================")

True Kernel: ((Constant({'variance': 1.185}) * Constant({'variance': 0.849})) * Linear({'variances': 1.288})) Log Marginal Likelihood: 91.86569782585374 RMSE: 0.6586396031328083
No std (0.0) in the true kernel
[Step 1] BIC: -175.13 LL: 90.97 | Linear({'variances': 1.0})
[Step 2] BIC: -171.73 LL: 90.97 | (Linear({'variances': 1.0}) * Constant({'variance': 1.0}))
[Step 3] BIC: -175.13 LL: 90.97 | Linear({'variances': 1.0})
[Step 4] BIC: -171.73 LL: 90.97 | (Linear({'variances': 1.0}) * Constant({'variance': 1.0}))
[Step 5] BIC: -175.13 LL: 90.97 | Linear({'variances': 1.0})
Greedy Search Kernel: Linear({'variances': 1.0}) Log Marginal Likelihood: 91.86569782583669 RMSE: 0.6586062908765267
No std (0.0) in the greedy kernel
 Basis kernel: RBF({'lengthscale': 1.0, 'variance': 1.0}) Log Marginal Likelihood: 85.35708041034214 RMSE: 0.6271722536401301
No std (0.0) in the kernel
 Basis kernel: Linear({'variances': 1.0}) Log Marginal Likelihood: 91.86569782583669 RMSE: 0.6586062908765267
No std 